In [1]:
import os
import gzip
import re
import polars as pl
import tarfile as tf
import shutil

In [3]:
def unzip(file_path, target_path, type='gz'):
    if type == 'gz':
        with gzip.open(file_path, 'rb') as gz_file, open(file_path[:-3], 'wb') as uncompressed_file:
            uncompressed_file.write(gz_file.read())
    elif type == 'tar':
        with tf.open(file_path, 'r') as tar_file:
            tar_file.extractall(file_path[:-4])
    else:
        raise ValueError(f'Unknown type: {type}')

In [8]:
def unzip_gz_files(directory_path):
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if filename.endswith('.gz'):
            unzip(file_path)
            print(f"Unzipped: {filename}")

In [13]:
def retrieve_characters_before_pattern(input_string):
    
    pattern = re.compile(r'^(.*?)(?=_)')

    match = re.search(pattern, input_string)

    if match:
        result = match.group(1)
        return result
    else:
        return None
print(retrieve_characters_before_pattern('abc_123'))

abc


In [18]:
def remove_characters_before_underscore(input_string):
    pattern = re.compile(r'^.*?_')
    result = re.sub(pattern, '', input_string)
    return result
print(remove_characters_before_underscore('abc_123'))

123


```
Sample_Name	Sample_Plate	Sample_Group	Pool_ID	Sentrix_ID	Sentrix_Position
1		                        Normal		        6026818108	R03C02
2		                        Normal		        6026818108	R03C02
3		                        Normal		        6026818135	R04C02
4		                        Normal		        6026818135	R04C02
5		                        Normal		        6026818144	R05C02
6		                        Normal		        6026818144	R05C02
7		                        Normal		        6026818135	R05C02
8		                        Normal		        6026818135	R05C02
9		                        Normal		        6026818139	R01C02
10		                        Normal		        6026818139	R01C02
```

In [69]:
normal_train = pl.read_csv("../data/train/sample_sheet.csv", skip_rows=7).filter(pl.col("Sample_Group") == "Normal")
normal_test = pl.read_csv("../data/test/sample_sheet.csv", skip_rows=7).filter(pl.col("Sample_Group") == "Normal")
series450 = pl.read_csv('../external_result/450k_series.csv', has_header=False).transpose().to_series()
series850 = pl.read_csv('../external_result/850k_series.csv', has_header=False).transpose().to_series()

In [34]:
# count of 450k series and 850k series
print(len(series450))
print(len(series850))

52
71


In [57]:
normal_train = normal_train.transpose()
normal_train

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_57,column_58,column_59,column_60,column_61,column_62,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""1""","""2""","""3""","""4""","""5""","""6""","""7""","""8""","""9""","""10""","""11""","""12""","""13""","""14""","""15""","""16""","""17""","""18""","""19""","""20""","""21""","""22""","""23""","""24""","""25""","""26""","""27""","""28""","""29""","""30""","""31""","""32""","""33""","""34""","""35""","""36""","""37""",…,"""58""","""59""","""60""","""61""","""62""","""63""","""64""","""65""","""66""","""67""","""68""","""69""","""70""","""71""","""72""","""73""","""74""","""75""","""76""","""77""","""78""","""79""","""80""","""81""","""82""","""83""","""84""","""85""","""86""","""87""","""88""","""89""","""90""","""91""","""92""","""93""","""94"""
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""",…,"""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal"""
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""6026818108""","""6026818108""","""6026818135""","""6026818135""","""6026818144""","""6026818144""","""6026818135""","""6026818135""","""6026818139""","""6026818139""","""6026818144""","""6026818144""","""6026818129""","""6026818129""","""6026818129""","""6026818129""","""6026818135""","""6026818135""","""6026818128""","""6026818128""","""6026818132""","""6026818132""","""6026818144""","""6026818144""","""6026818144""","""6026818144""","""6026818145""","""6026818145""","""6026818129""","""6026818129""","""6026818055""","""6026818055""","""6026818102""","""6026818102""","""6026818122""","""60268

In [59]:
directory_path = '../data_TNBC/mixed/'
# df450k = pl.DataFrame({'Sample_Name': [], 'Sample_Plate': [], 'Sample_Group': [], 'Pool_ID': [], 'Sentrix_ID': [], 'Sentrix_Position': []})
# df850k = pl.DataFrame({'Sample_Name': [], 'Sample_Plate': [], 'Sample_Group': [], 'Pool_ID': [], 'Sentrix_ID': [], 'Sentrix_Position': []})

# df450k = pl.DataFrame({"": [0,"","","","",""]})
# df450k = pl.DataFrame(
# df850k = pl.DataFrame({"": [0,"","","","",""]})
i450k = 94
i850k = 0
for filename in os.listdir(directory_path):
    gsm_id, Sentrix_ID, Sentrix_Position, endstr = filename.split('_')
    # print(gsm_id, Sentrix_ID, Sentrix_Position, endstr)
    if gsm_id in series450:
        # shutil.copyfile(os.path.join(directory_path, filename), f'../data_TNBC/450k/{Sentrix_ID}_{Sentrix_Position}_{endstr}')
        # create series of 450k of one sample
        tmp = pl.Series(str(i450k),[str(i450k+1), "", "tumor_TNBC", "", Sentrix_ID, Sentrix_Position])
        normal_train.insert_column(i450k, tmp)
        i450k += 1
    elif gsm_id in series850:
        # shutil.copyfile(os.path.join(directory_path, filename), f'../data_TNBC/850k/{Sentrix_ID}_{Sentrix_Position}_{endstr}')
        # tmp = pl.Series(str(i850k),[i850k+1, "", "tumor_TNBC", "", Sentrix_ID, Sentrix_Position])
        # df850k.insert_column(i850k, tmp)
        i850k += 1
    else:
        print("unexpected error")
        break
# df450k.to_csv('../data_TNBC/450k/sample_sheet.csv')
# df850k.to_csv('../data_TNBC/850k/sample_sheet.csv')

In [47]:
# df450k.transpose().fill_null(0).write_csv('../data_TNBC/450k/sample_sheet.csv')

In [49]:
# df850k.transpose().fill_null(0).write_csv('../data_TNBC/850k/sample_sheet.csv')

In [73]:
normal_train = normal_train.transpose()

In [66]:
normal_train.columns = ['Sample_Name', 'Sample_Plate', 'Sample_Group', 'Pool_ID', 'Sentrix_ID', 'Sentrix_Position']
normal_train

Sample_Name,Sample_Plate,Sample_Group,Pool_ID,Sentrix_ID,Sentrix_Position
str,str,str,str,str,str
"""1""",null,"""Normal""",null,"""6026818108""","""R03C02"""
"""2""",null,"""Normal""",null,"""6026818108""","""R03C02"""
"""3""",null,"""Normal""",null,"""6026818135""","""R04C02"""
"""4""",null,"""Normal""",null,"""6026818135""","""R04C02"""
"""5""",null,"""Normal""",null,"""6026818144""","""R05C02"""
"""6""",null,"""Normal""",null,"""6026818144""","""R05C02"""
"""7""",null,"""Normal""",null,"""6026818135""","""R05C02"""
"""8""",null,"""Normal""",null,"""6026818135""","""R05C02"""
"""9""",null,"""Normal""",null,"""6026818139""","""R01C02"""


In [68]:
normal_train.write_csv('../data_TNBC/450k/sample_sheet.csv')

In [79]:
normal_train

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_57,column_58,column_59,column_60,column_61,column_62,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""1""","""2""","""3""","""4""","""5""","""6""","""7""","""8""","""9""","""10""","""11""","""12""","""13""","""14""","""15""","""16""","""17""","""18""","""19""","""20""","""21""","""22""","""23""","""24""","""25""","""26""","""27""","""28""","""29""","""30""","""31""","""32""","""33""","""34""","""35""","""36""","""37""",…,"""58""","""59""","""60""","""61""","""62""","""63""","""64""","""65""","""66""","""67""","""68""","""69""","""70""","""71""","""72""","""73""","""74""","""75""","""76""","""77""","""78""","""79""","""80""","""81""","""82""","""83""","""84""","""85""","""86""","""87""","""88""","""89""","""90""","""91""","""92""","""93""","""94"""
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""",…,"""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal""","""Normal"""
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""6026818108""","""6026818108""","""6026818135""","""6026818135""","""6026818144""","""6026818144""","""6026818135""","""6026818135""","""6026818139""","""6026818139""","""6026818144""","""6026818144""","""6026818129""","""6026818129""","""6026818129""","""6026818129""","""6026818135""","""6026818135""","""6026818128""","""6026818128""","""6026818132""","""6026818132""","""6026818144""","""6026818144""","""6026818144""","""6026818144""","""6026818145""","""6026818145""","""6026818129""","""6026818129""","""6026818055""","""6026818055""","""6026818102""","""6026818102""","""6026818122""","""60268

In [80]:
normal_list = [f"{col[4]}_{col[5]}" for col in normal_train.iter_columns()]


In [81]:
normal_list

['6026818108_R03C02',
 '6026818108_R03C02',
 '6026818135_R04C02',
 '6026818135_R04C02',
 '6026818144_R05C02',
 '6026818144_R05C02',
 '6026818135_R05C02',
 '6026818135_R05C02',
 '6026818139_R01C02',
 '6026818139_R01C02',
 '6026818144_R06C02',
 '6026818144_R06C02',
 '6026818129_R01C01',
 '6026818129_R01C01',
 '6026818129_R03C02',
 '6026818129_R03C02',
 '6026818135_R05C01',
 '6026818135_R05C01',
 '6026818128_R06C01',
 '6026818128_R06C01',
 '6026818132_R02C01',
 '6026818132_R02C01',
 '6026818144_R06C01',
 '6026818144_R06C01',
 '6026818144_R01C02',
 '6026818144_R01C02',
 '6026818145_R03C01',
 '6026818145_R03C01',
 '6026818129_R04C01',
 '6026818129_R04C01',
 '6026818055_R02C01',
 '6026818055_R02C01',
 '6026818102_R03C02',
 '6026818102_R03C02',
 '6026818122_R01C02',
 '6026818122_R01C02',
 '6026818122_R05C02',
 '6026818122_R05C02',
 '6026818111_R01C01',
 '6026818111_R01C01',
 '6026818111_R06C01',
 '6026818111_R06C01',
 '6026818111_R04C02',
 '6026818111_R04C02',
 '6026818111_R05C02',
 '60268181

In [87]:
for filename in os.listdir("../data/train/") :
    if filename.endswith('.idat'):
        Sentrix_ID, Sentrix_Position, endstr = filename.split('_')
        if f"{Sentrix_ID}_{Sentrix_Position}" in normal_list :
            shutil.copyfile(os.path.join("../data/train/", filename), f'../data_TNBC/normal/{filename}')
            print(f"copied: {filename}")

copied: 6026818055_R02C01_Grn.idat
copied: 6026818055_R02C01_Red.idat
copied: 6026818070_R01C01_Grn.idat
copied: 6026818070_R01C01_Red.idat
copied: 6026818102_R03C02_Grn.idat
copied: 6026818102_R03C02_Red.idat
copied: 6026818108_R03C02_Grn.idat
copied: 6026818108_R03C02_Red.idat
copied: 6026818111_R01C01_Grn.idat
copied: 6026818111_R01C01_Red.idat
copied: 6026818111_R04C02_Grn.idat
copied: 6026818111_R04C02_Red.idat
copied: 6026818111_R05C02_Grn.idat
copied: 6026818111_R05C02_Red.idat
copied: 6026818111_R06C01_Grn.idat
copied: 6026818111_R06C01_Red.idat
copied: 6026818122_R01C02_Grn.idat
copied: 6026818122_R01C02_Red.idat
copied: 6026818122_R05C02_Grn.idat
copied: 6026818122_R05C02_Red.idat
copied: 6026818128_R06C01_Grn.idat
copied: 6026818128_R06C01_Red.idat
copied: 6026818129_R01C01_Grn.idat
copied: 6026818129_R01C01_Red.idat
copied: 6026818129_R03C02_Grn.idat
copied: 6026818129_R03C02_Red.idat
copied: 6026818129_R04C01_Grn.idat
copied: 6026818129_R04C01_Red.idat
copied: 6026818132_R